# Imports

In [1]:
import pandas as pd
from unidecode import unidecode
import dtale

# Load Data

In [2]:
df_gupy = pd.read_excel('../data/data_raw/vagas_gupy.xlsx')

df_vagas = pd.concat([df_gupy], axis = 0)
df_vagas.head()

,site_da_vaga,link,data_publicacao,data_coleta,titulo_da_vaga,local,modalidade,contrato,acessibilidade,nome_empresa,codigo_vaga,descricao,regime
0,Gupy,https://softplan.gupy.io/job/eyJqb2JJZCI6NjYxO...,Publicada em: 22/01/2024,2024-01-22,Pessoa Analista de Dados Jr - Cód.9734,Não informado,Remoto,Efetivo,Também p/ PcD,Softplan,NaN,Descrição da vagaSomos um dos maiores ecossist...,NaN
1,Gupy,https://cielo.gupy.io/job/eyJqb2JJZCI6NjY0NTI0...,Publicada em: 22/01/2024,2024-01-22,Analista de Engenharia de Dados,Barueri - SP,Híbrido,Efetivo,NaN,Cielo,NaN,Descrição da vagaTrabalhando na área de Sistem...,NaN
2,Gupy,https://barcelos.gupy.io/job/eyJqb2JJZCI6NjY0O...,Publicada em: 22/01/2024,2024-01-22,ANALISTA DE INTELIGÊNCIA DE DADOS/ CAMPOS DOS ...,Campos dos... - RJ,Presencial,Efetivo,Também p/ PcD,Barcelos,NaN,Descrição da vagaBuscamos gente que compartilh...,NaN
3,Gupy,https://farmaciasapp.gupy.io/job/eyJqb2JJZCI6N...,Publicada em: 22/01/2024,2024-01-22,Analista de Dados PL,Não informado,Remoto,Efetivo,NaN,Farmácias APP,NaN,Descrição da vagaPensou em farmácia? Farmácias...,NaN
4,Gupy,https://lojastorra.gupy.io/job/eyJqb2JJZCI6NjY...,Publicada em: 22/01/2024,2024-01-22,ANALISTA DE DADOS - CRM - BI,São Paulo - SP,Presencial,Efetivo,Também p/ PcD,Lojas Torra,NaN,Descrição da vagaDESCRIÇÃO DA VAGAVocê já pens...,NaN


# Features

## Senioridade

In [28]:
# matches para as senioridades
junior_matches = ['junior', ' jr' , ' i ', ' l']
pleno_matches = ['pleno', ' pl', ' ii', ' ll']
senior_matches = ['senior', ' sr', ' iii', 'lll']

# removendo acentos e deixando letras minúsculas
df_vagas['titulo_da_vaga_tratado'] = df_vagas['titulo_da_vaga'].apply(lambda x: unidecode(x.lower()))

# função de faz matches com as senioridades das vagas
def busca_senioridade(titulo_vaga, match_list):
    senioridade = False
    for sub_string in match_list:
        if sub_string in titulo_vaga:
            senioridade = True
    
    return senioridade

# aplicando função
df_vagas['senioridade_junior'] = df_vagas['titulo_da_vaga_tratado'].apply(lambda titulo_vaga: busca_senioridade(titulo_vaga, junior_matches))
df_vagas['senioridade_pleno'] = df_vagas['titulo_da_vaga_tratado'].apply(lambda titulo_vaga: busca_senioridade(titulo_vaga, pleno_matches))
df_vagas['senioridade_senior'] = df_vagas['titulo_da_vaga_tratado'].apply(lambda titulo_vaga: busca_senioridade(titulo_vaga, senior_matches))

# categorizando senioridades
df_vagas['senioridade'] = df_vagas[['senioridade_junior', 
                                    'senioridade_pleno', 
                                    'senioridade_senior']].apply(lambda x:  'Pleno/Sênior' if (x['senioridade_pleno']) and (x['senioridade_senior']) else
                                                                            'Júnior/Pleno' if (x['senioridade_junior']) and (x['senioridade_pleno']) else
                                                                            'Júnior' if x['senioridade_junior'] else 
                                                                            'Pleno' if x['senioridade_pleno'] else
                                                                            'Sênior' if x['senioridade_senior'] else
                                                                            'Não informado'
                                                                            , axis = 1)

In [29]:
df_vagas.loc[df_vagas['senioridade'] == 'Júnior', 'titulo_da_vaga_tratado'].unique()

array(['pessoa analista de dados jr - cod.9734', 'analista de dados jr.',
       'analista de dados junior', 'analista de dados junior ',
       'analista de dados junior - prazo determinado | performance de tecnologia',
       'analista de dados - junior', 'analista de dados junior - centro',
       'analista de dados junior | preco e inventario',
       'analista de dados junior - petz holding  ',
       'analista de dados jr', 'analista de dados jr ',
       'cas | analista de dados jr - foco agro',
       'analista de dados jr (area supply chain)',
       'analista de dados | lapa de baixo/sp | afirmativa pessoas com deficiencia',
       'analista de dados estrategicos jr',
       'analista de arquitetura de dados jr ',
       'analista de dados junior - jandira - sp', 'analista dados jr',
       'nv | analista de dados i ',
       'analista de dados jr - santa maria/df',
       'analista de banco de dados jr (sql/ power bi)',
       'analista de inteligencia de dados jr',
       '

## Localidade

In [30]:
df_vagas['cidade'] = df_vagas['local'].apply(lambda x: 'Não informado' if x == 'Não informado' else x.split(' - ')[0])
df_vagas['estado'] = df_vagas['local'].apply(lambda x: 'Não informado' if x == 'Não informado' else x.split(' - ')[1])

In [31]:
df_vagas['cidade']

0      Não informado
1            Barueri
2      Campos dos...
3      Não informado
4          São Paulo
           ...      
194    Rio de Jan...
195        São Paulo
196    Rio de Jan...
197    Não informado
198        São Paulo
Name: cidade, Length: 199, dtype: object

# Save Dataframe

In [32]:
columns_selected = [
'site_da_vaga',
 'link',
 'data_publicacao',
 'data_coleta',
 'titulo_da_vaga',
 'senioridade',
 'cidade',
 'estado',
 'modalidade',
 'contrato',
 'acessibilidade',
 'nome_empresa',
 'codigo_vaga',
 'descricao'
]

In [33]:
df_vagas[columns_selected].to_excel('vagas_clean.xlsx', index=False)

In [25]:
df_vagas[columns_selected]

,site_da_vaga,link,data_publicacao,data_coleta,titulo_da_vaga,senioridade,cidade,estado,modalidade,contrato,acessibilidade,nome_empresa,codigo_vaga,descricao
0,Gupy,https://softplan.gupy.io/job/eyJqb2JJZCI6NjYxO...,Publicada em: 22/01/2024,2024-01-22,Pessoa Analista de Dados Jr - Cód.9734,Júnior,Não informado,Não informado,Remoto,Efetivo,Também p/ PcD,Softplan,NaN,Descrição da vagaSomos um dos maiores ecossist...
1,Gupy,https://cielo.gupy.io/job/eyJqb2JJZCI6NjY0NTI0...,Publicada em: 22/01/2024,2024-01-22,Analista de Engenharia de Dados,Não informado,Barueri,SP,Híbrido,Efetivo,NaN,Cielo,NaN,Descrição da vagaTrabalhando na área de Sistem...
2,Gupy,https://barcelos.gupy.io/job/eyJqb2JJZCI6NjY0O...,Publicada em: 22/01/2024,2024-01-22,ANALISTA DE INTELIGÊNCIA DE DADOS/ CAMPOS DOS ...,Júnior,Campos dos...,RJ,Presencial,Efetivo,Também p/ PcD,Barcelos,NaN,Descrição da vagaBuscamos gente que compartilh...
3,Gupy,https://farmaciasapp.gupy.io/job/eyJqb2JJZCI6N...,Publicada em: 22/01/2024,2024-01-22,Analista de Dados PL,Pleno,Não informado,Não informado,Remoto,Efetivo,NaN,Farmácias APP,NaN,Descrição da vagaPensou em farmácia? Farmácias...
4,Gupy,https://lojastorra.gupy.io/job/eyJqb2JJZCI6NjY...,Publicada em: 22/01/2024,2024-01-22,ANALISTA DE DADOS - CRM - BI,Não informado,São Paulo,SP,Presencial,Efetivo,Também p/ PcD,Lojas Torra,NaN,Descrição da vagaDESCRIÇÃO DA VAGAVocê já pens...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
194,Gupy,https://infotecbrasil.gupy.io/job/eyJqb2JJZCI6...,Publicada em: 26/12/2022,2024-01-22,Analista de Dados - Home Office,Não informado,Rio de Jan...,RJ,Remoto,Efetivo,Também p/ PcD,Infotec Brasil,NaN,Descrição da vagaExecutar atividades de suport...
195,Gupy,https://gruposkytech&76015&inactive.gupy.io/jo...,Publicada em: 31/10/2022,2024-01-22,Analista de dados,Não informado,São Paulo,SP,Presencial,Efetivo,NaN,Grupo Skytech,NaN,NaN
196,Gupy,https://brivia.gupy.io/job/eyJqb2JJZCI6OTQ5ODA...,Publicada em: 08/06/2021,2024-01-22,Analista de Dados,Não informado,Rio de Jan...,RJ,Remoto,Banco de Talentos,Também p/ PcD,Brivia,NaN,Descrição da vagaVaga para nosso BANCO DE TALE...
197,Gupy,https://queroserglobalsys.gupy.io/job/eyJqb2JJ...,Publicada em: 04/06/2021,2024-01-22,Banco de Talentos - Analista de Dados/ Analist...,Não informado,Não informado,Não informado,Banco de Talentos,Também p/ PcD,NaN,Globalsys Soluções em TI,NaN,NaN


In [26]:
d = dtale.show(df_vagas)
d.open_browser()

In [ ]:
df_vagas[['senioridade', 'link']].groupby('senioridade').count().reset_index().sort_values('link', ascending=False)

,senioridade,link
1,Não informado,78
4,Sênior,42
0,Júnior,37
2,Pleno,37
3,Pleno/Sênior,5


2024-01-22 20:29:09,630 - INFO     - Executing shutdown due to inactivity...
2024-01-22 20:29:09,740 - INFO     - Executing shutdown...
2024-01-22 20:29:09,743 - INFO     - Not running with the Werkzeug Server, exiting by searching gc for BaseWSGIServer
